# UWavy Navigation - Solution

1. Overview
2. Inertial Navigation
3. Image Recognition
4. Synthesis
5. Conclusion


## Overview (TODO)



## Inertial Navigation

### Generating a "True" Path
[TODO: Animation]

In [ ]:
def wide_uturn(leg=10):
    ones = np.ones((leg,1));
    zeros = np.zeros((leg,1));
    
    leg1 = np.hstack((ones, zeros))
    leg2 = np.hstack((zeros, ones))
    leg3 = np.hstack((-ones, zeros))
    return np.concatenate((leg1, leg2, leg3), axis=0)

def s_turn(short_leg=5):
    ones = np.ones((short_leg,1));
    zeros = np.zeros((short_leg,1));
    
    leg1 = np.hstack((ones, zeros))
    leg2 = np.hstack((zeros*2, -ones*2))
    leg3 = np.hstack((ones, zeros))
    return np.concatenate((leg1, leg2, leg3), axis=0)

# Set out a path, starting at START_POS and following updates along DELTAS_TRUE
start_pos = np.array([[1500,850]])
deltas_true = (np.vstack((np.vstack([s_turn(1) for _ in range(3)]), wide_uturn(3))) * 25).astype(np.float64)
path_true = np.cumsum(np.vstack((start_pos, deltas_true)), axis=0)

## TODO Plot path_true
ani = FlightAnimator()
ani.add_path(path_true, label="True Path")
ani.start();

### Introducing Systematic and Random Error
[TODO: Animation]

In [ ]:
# Simulate some input from the onboard sensors and camera
deltas_inert = (deltas_true - np.array([3, -1])).astype(np.float64) # Add systematically biased error 
# TODO add random error
ani = FlightAnimator()
ani.add_path(path_sim, label="Simulated Path")
ani.start();

## Image Recognition

## 

### Simulating the onboard camera

In [1]:
ref_img = load_image()
img = take_picture(ref_img)
plot(ref_img, img)
# TODO subplots, side-by-side etc

NameError: name 'load_image' is not defined

### Simulating the image database

In [2]:
# TODO: read_img from PIL, or something



### Searching the database for the captured image

In [3]:
# TODO: heatmaps, etc



## Synthesis

### Informing the Search Algorithm
1. Starting Point = from intertial nav
1. Sprial Pattern
1. Search Confidence
### Combining Intertial + Image Recognition
1. Putting the pieces together
[Animation with both inertial + img rec]

In [ ]:
ani = FlightAnimation()
ani.add_target_start(path_true[0])
ani.add_target_end(path_true[-1])
ani.add_path(path_sim)
ani.add_path(path_sim_plus)
ani.start();

## Conclusion
[TODO: Make some over-generalizing statements]
### Future Steps
1. Increasing search radius dynamically